In [1]:
import mrcfile
import numpy as np
from subprocess import call
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
import os
import fnmatch
from ClassFiles.ut import create_single_folder
from ClassFiles.Framework import AdversarialRegulariser
import random

In [2]:
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name).replace("\\", "/"))
    return result

In [3]:
path = '/local/scratch/public/sl767/MRC_Data/org/'
path_adv = '/local/scratch/public/sl767/MRC_Data/Data_002_10k/SGD'

In [4]:
train_list = find('*it300_class001.mrc*', path_adv)
train_amount = len(train_list)
print('Training Pictures found: ' + str(train_amount))
eval_list = ''
eval_amount = len(eval_list)
print('Evaluation Pictures found: ' + str(eval_amount))

Training Pictures found: 1
Evaluation Pictures found: 0


In [5]:
def locate_gt(adversarial_path):
    pdb_id = adversarial_path[-31:-27]
    l = find('*'+pdb_id+'.mrc', path)
    if not len(l)==1:
        raise ValueError('non-unique pdb id: '+l)
    else:
        return l[0]

In [6]:
def get_image(number, training):
    if training:
        L = train_list
    else:
        L = eval_list
    adv_path=L[number]
    with mrcfile.open(adv_path) as mrc:
        adv = mrc.data
    with mrcfile.open(locate_gt(adv_path)) as mrc:
        gt = mrc.data
    return gt, adv

def load_data(training_data=True):
    if training_data:
        n = train_amount
    else:
        n = eval_amount
    return get_image(random.randint(0, n-1), training=training_data)

In [7]:
BATCH_SIZE=1
def get_batch(training_data=True):
    true = np.zeros(shape=(BATCH_SIZE, 96,96,96))
    adv = np.zeros(shape=(BATCH_SIZE, 96,96,96))
    for k in range(BATCH_SIZE):
        gt, adver = load_data(training_data=training_data)
        true[k,...]=gt
        adv[k,...]=adver
    return gt,adv

In [8]:
saves_path = '/local/scratch/public/sl767/SPA/Saves/Adversarial_Regulariser/SGD_Trained'
regularizer = AdversarialRegulariser(saves_path)

No save found


In [9]:
def evaluate():
    gt, adv = get_batch(training_data=True)
    regularizer.test(groundTruth=gt, adversarial=adv, fourier_data =False)

LEARNING_RATE=0.001
def train(steps):
    for k in range(steps):
        gt, adv = get_batch()
        regularizer.train(groundTruth=gt, adversarial=adv, learning_rate=LEARNING_RATE, fourier_data =False)
        if k%10==0:
            evaluate()
    regularizer.save()

In [10]:
for k in range(5):
    train(100)

Progress saved
Progress saved
Progress saved
Progress saved
Progress saved
